In [18]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pygal
from pygal.style import Style
from textblob import TextBlob

nltk.download('punkt')
nltk.download('stopwords')

data = pd.read_csv("csv/tweets.csv", sep = ',')

[nltk_data] Downloading package punkt to /home/nomdebrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nomdebrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
columns_to_use = ['created_str','text']
data = data[columns_to_use]
data = data.dropna(subset=['text'])

#convert created_str to datetime and drop hour, min, sec
data['created_str'] = pd.to_datetime(data['created_str'])
data.created_str = data.created_str.map(lambda x: x.replace(day=1, hour=0, minute=0, second=0))


clintonData = data[data['text'].str.contains('clinton|hillary', case=False)]
trumpData = data[data['text'].str.contains('trump|donald', case=False)]
bothData = data[data['text'].str.contains('clinton|hillary|trump|donald', case=False)]

#add polarity and subjectivity columns to dataframes
clintonData['polarity'] = clintonData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
clintonData['subjectivity'] = clintonData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
trumpData['polarity'] = trumpData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
trumpData['subjectivity'] = trumpData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
bothData['polarity'] = bothData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
bothData['subjectivity'] = bothData['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)

df_C1 = clintonData.groupby('created_str').mean()
df_T1 = trumpData.groupby('created_str').mean()
df_B1 = bothData.groupby('created_str').mean()

print(data.shape)
print(clintonData.shape)
print(trumpData.shape)
print(bothData.shape)

/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



(203430, 2)
(25555, 4)
(40403, 4)
(57521, 4)


/home/nomdebrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
#get dates back into column instead of being an index
df_C1.to_csv('csv/clinton_1.csv', sep=',')
df_C1 = pd.read_csv('csv/clinton_1.csv')

df_T1.to_csv('csv/trump_1.csv', sep=',')
df_T1 = pd.read_csv('csv/trump_1.csv')

df_B1.to_csv('csv/both_1.csv', sep=',')
df_B1 = pd.read_csv('csv/both_1.csv')

In [4]:
custom_style = Style(colors=('red','blue','purple'))
date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Polarity Over Time'
date_chart.x_labels = df_C1['created_str']
date_chart.add("trump", df_T1['polarity'])
date_chart.add("Clinton", df_C1['polarity'])
date_chart.add("Both", df_B1['polarity'])
date_chart.render_to_file('images/polarity_date.svg')

date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Subjectivity Over Time'
date_chart.x_labels = df_C1['created_str']
date_chart.add("trump", df_T1['subjectivity'])
date_chart.add("Clinton", df_C1['subjectivity'])
date_chart.add("Both", df_B1['subjectivity'])
date_chart.render_to_file('images/subjectivity_date.svg')

In [5]:
custom_style = Style(colors=('red',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'trump'
gauge.value_formatter = percent_formatter
gauge.add('trump', [{'value': round(len(trumpData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/t_guage.svg')

custom_style = Style(colors=('blue',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'Clinton'
gauge.value_formatter = percent_formatter
gauge.add('Clinton', [{'value': round(len(clintonData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/c_guage.svg')

custom_style = Style(colors=('purple',))
gauge = pygal.SolidGauge(inner_radius=0.70, style=custom_style, show_legend=False)
percent_formatter = lambda x: '{:.10g}%'.format(x)
gauge.title = 'Both'
gauge.value_formatter = percent_formatter
gauge.add('Both', [{'value': round(len(bothData)/len(data)*100,1), 'max_value': 100}])
gauge.render_to_file('images/b_guage.svg')

In [9]:
#reindex dataframe
bothData = bothData.reset_index(drop=True)

<h3>Example Tweets</h3>

In [10]:
clintonData = clintonData.sort_values(by=['polarity'])
clintonData = clintonData.reset_index(drop=True)
print(clintonData.loc[0:19,['text','polarity']])

                                                 text  polarity
0   RT @luvGodncountry: Wikileaks: Clinton Adviser...      -1.0
1   RT @VivaLaAmes: Holy CRAP, HILLARY HAS LOST IT...      -1.0
2   RT @amborin: We must ensure that Hillary is no...      -1.0
3   Clinton says Trump has been ‘degrading, insult...      -1.0
4   RT @ElianaBenador: Exclusive — Trumpocrats Dem...      -1.0
5   RT @JaredWyand: You are disgusting @HillaryCli...      -1.0
6   RT @1Kimsey: @Crimsonmile @RichardGrenell @tam...      -1.0
7   RT @pmbasse: Team Hillary Gave GOP Establishme...      -1.0
8   RT @Harlan: Hillary is running a nasty, divisi...      -1.0
9   RT @OzzeeLady: BREAKING: Federal Judge Gives H...      -1.0
10  RT @LindaSuhler: But TRUMP is unfit?\r\nFrom t...      -1.0
11  RT @ajenable: Embrace the power of and, my fri...      -1.0
12  RT @trumpology: It's outrageous. They can keep...      -1.0
13  RT @LouDobbs: Irony, Anyone? "Sometimes Bill A...      -1.0
14  RT @jenncox24: @RacySicilian @magnif

In [11]:
trumpData = trumpData.sort_values(by=['subjectivity'])
trumpData = trumpData.reset_index(drop=True)
print(trumpData.loc[0:19,['text','subjectivity']])

                                                 text  subjectivity
0   RT @mcicero10: #BernieSanders #Trump people sh...           0.0
1   RT @Stonewall_77: Rewards For Services Rendere...           0.0
2   RT @myGianLuca: RT @hillaryclinton @billclinto...           0.0
3   RT @ForecasterEnten: Trump 46, Cruz 31, Rubio ...           0.0
4   RT @TrivWorks: • Trump\r\n• Celebrity Deaths \...           0.0
5   RT @micafarha: @LeahR77 @Braveheart_USA @dreww...           0.0
6   RT @realDailyWire: Ep. 220 - Trump Saved Jobs!...           0.0
7   RT @Born_To_DYE: #TrumpsFavoriteHeadline Putin...           0.0
8   RT @darleneturner53: For Trump to release his ...           0.0
9   Trump Jr. likens Syrian refugees to Skittles h...           0.0
10         Even #ChildrenThinkThat Trump is immature.           0.0
11  RT @MIPooh: Trump Ag Dept @USDA Orders Hiding ...           0.0
12  The Anti-Inauguration in a nutshell: we are of...           0.0
13  RT @TeaPartyOrg: Trump Cracks the Electoral 

<h3>Tokenize Tweets</h3>

In [13]:
clintonData['tokenized_sents'] = clintonData.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
trumpData['tokenized_sents'] = trumpData.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
bothData['tokenized_sents'] = bothData.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

<h3>Lemmatize Tweets</h3>

In [14]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

clintonData['text_lemmatized'] = clintonData.tokenized_sents.apply(lemmatize_text)
trumpData['text_lemmatized'] = trumpData.tokenized_sents.apply(lemmatize_text)
bothData['text_lemmatized'] = bothData.tokenized_sents.apply(lemmatize_text)

<h3>Remove Stop Words From Tweets</h3>

In [24]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
custom_stopwords = ['RT','https','http','@',':']
stop.extend(custom_stopwords)

clintonData['without_stopwords'] = clintonData.text_lemmatized.apply(lambda x: [item for item in x if item not in stop])
trumpData['without_stopwords'] = trumpData.text_lemmatized.apply(lambda x: [item for item in x if item not in stop])
bothData['without_stopwords'] = bothData.text_lemmatized.apply(lambda x: [item for item in x if item not in stop])

0        luvGodncountry : Wikileaks : Clinton Adviser T...
1        VivaLaAmes : Holy CRAP , HILLARY HAS LOST IT !...
2        amborin : We must ensure Hillary elected . She...
3        Clinton say Trump ha ‘ degrading , insulting a...
4        ElianaBenador : Exclusive — Trumpocrats Demand...
5        JaredWyand : You disgusting HillaryClinton . T...
6        1Kimsey : Crimsonmile RichardGrenell tamaragit...
7        pmbasse : Team Hillary Gave GOP Establishment ...
8        Harlan : Hillary running nasty , divisive camp...
9        OzzeeLady : BREAKING : Federal Judge Gives Hil...
10       LindaSuhler : But TRUMP unfit ? From worst POT...
11       ajenable : Embrace power , friend . Hillary Do...
12       trumpology : It 's outrageous . They keep dump...
13       LouDobbs : Irony , Anyone ? `` Sometimes Bill ...
14       jenncox24 : RacySicilian magnifier661 HillaryC...
15       NolteNC : Trump end Birtherism Hillary birthed...
16       lesliermyers : GOP created Trump disgusting li.

<h3>Join tokens back together</h3>

In [42]:
clintonData['processed'] = clintonData.without_stopwords.apply(lambda x: ' '.join(word for word in x))
trumpData['processed'] = trumpData.without_stopwords.apply(lambda x: ' '.join(word for word in x))
bothData['processed'] = bothData.without_stopwords.apply(lambda x: ' '.join(word for word in x))

In [43]:
#add polarity and subjectivity columns to dataframes
clintonData['polarity_2'] = clintonData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
clintonData['subjectivity_2'] = clintonData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
trumpData['polarity_2'] = trumpData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
trumpData['subjectivity_2'] = trumpData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
bothData['polarity_2'] = bothData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
bothData['subjectivity_2'] = bothData['processed'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)


df_C2 = clintonData.groupby('created_str').mean()
df_T2 = trumpData.groupby('created_str').mean()
df_B2 = bothData.groupby('created_str').mean()


In [44]:
#get dates back into column instead of being an index
df_C2.to_csv('csv/clinton_2.csv', sep=',')
df_C2 = pd.read_csv('csv/clinton_2.csv')

df_T2.to_csv('csv/trump_2.csv', sep=',')
df_T2 = pd.read_csv('csv/trump_2.csv')

df_B2.to_csv('csv/both_2.csv', sep=',')
df_B2 = pd.read_csv('csv/both_2.csv')

In [45]:
custom_style = Style(colors=('red','blue','purple'))
date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Polarity Over Time'
date_chart.x_labels = df_C2['created_str']
date_chart.add("trump", df_T2['polarity_2'])
date_chart.add("Clinton", df_C2['polarity_2'])
date_chart.add("Both", df_B2['polarity_2'])
date_chart.render_to_file('images/polarity_date_processed.svg')

date_chart = pygal.Line(x_label_rotation=60, fill=False, style=custom_style)
date_chart.title = 'Subjectivity Over Time'
date_chart.x_labels = df_C2['created_str']
date_chart.add("trump", df_T2['subjectivity_2'])
date_chart.add("Clinton", df_C2['subjectivity_2'])
date_chart.add("Both", df_B2['subjectivity_2'])
date_chart.render_to_file('images/subjectivity_date_processed.svg')